## <font color="red">Caution!</font>

Before running a Marlowe contract on `mainnet`, it is wise to do the following in order to avoid losing funds:

1. Understand the [Marlowe Language](https://marlowe.iohk.io/).
2. Understand Cardano\'s [Extended UTxO Model](https://docs.cardano.org/learn/eutxo-explainer).
3. Read and understand the [Marlowe Best Practices Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/best-practices.md).
4. Read and understand the [Marlowe Security Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/security.md).
5. Use [Marlowe Playground](https://play.marlowe.iohk.io/) to flag warnings, perform static analysis, and simulate the contract.
6. Use [Marlowe CLI\'s](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe-cli/ReadMe.md) `marlowe-cli run analyze` tool to study whether the contract can run on a Cardano network.
7. Run *all execution paths* of the contract on a [Cardano testnet](https://docs.cardano.org/cardano-testnet/overview).

---

# The ACTUS Principal at Maturity (PAM) Contract

The ACTUS PAM contract is simply a loan with periodic interest payments on a fixed schedule and a final payment of the principal. For brevity, we demonstrate this contract with just one installment of interest, followed by the repayment of the principal.

The contract will be created from a template using the `marlowe-cli template actus` command, but it could also be created in the [Marlowe Playground](https://marlowe-playground-staging.plutus.aws.iohkdev.io/) or any other Marlowe-contract design tool that can output a Marlowe contract in JSON format. The contract looks like the following in Blockly format:

![The ACTUS PAM contract](actus-pam-1.png)

## Set Up

Use `mainnet`.

In [1]:
. ../../mainnet.env

Use the standard example roles.

In [2]:
. ../../dramatis-personae/roles.env

## Role tokens

This contract uses [Ada Handles](https://adahandle.com/) as role tokens:
- Christopher Marlowe = [\$c.marlowe](https://pool.pm/asset1z2xzfc6lu63jfmfffe2w3nyf6420eylv8e2xjp)
- John Webster = [\$j.webster](https://pool.pm/asset1zdcycnnmg6dx5dy030u4cu0zdn63r2scghg2p4)

*Note: Only use a pre-minted token as a Marlowe role if you have reviewed the monetary policy for security vulnerabilities.*

Here is the currency symbol for Ada handles on `mainnet`:

In [3]:
echo "ROLES_CURRENCY = $ROLES_CURRENCY"

ROLES_CURRENCY = f0ff48bbb7bbe9d59a40f1ce90e9e9d0ff5002ec48f232b49ca0fb9a


## Design the Contract

We set the parameters for the ACTUS PAM contract, first in environment variables and then in the JSON file that contains the full set of ACTUS parameters. See [marlowe-actus](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe-actus/README.md) for more information on using ACTUS in Marlowe.


The ACTUS PAM contract has two participants:
1. The *party* `c.marlowe` that lends the funds.
2. The *counterparty* `j.webster` that receives the loan and pays it back with interest.

In [4]:
SECOND=1000
MINUTE=$((60 * SECOND))
NOW="$(($(date -u +%s) * SECOND))"
echo "NOW = $(date -d @$((NOW / SECOND)))"

ADA=1000000
MINIMUM_ADA=$((3 * ADA))
echo "MINIMUM_ADA = $MINIMUM_ADA"

PRINCIPAL=100
INTEREST_RATE=0.02
INTEREST=$(jq -n $PRINCIPAL*$INTEREST_RATE)
echo "PRINCIPAL = $PRINCIPAL"
echo "INTEREST = $INTEREST"

STATUS_DATE=$(date -d "$(date -u -R -d @$((NOW/1000)))" +"%Y-%m-%dT00:00:00")
INITIAL_EXCHANGE_DATE=$(date -d "$(date -u -R -d @$((NOW/1000))) + 1 year" +"%Y-01-01T00:00:00")
MATURITY_DATE=$(date -d "$(date -u -R -d @$((NOW/1000))) + 2 year" +"%Y-01-01T00:00:00")

echo "STATUS_DATE = $STATUS_DATE"
echo "INITIAL_EXCHANGE_DATE = $INITIAL_EXCHANGE_DATE"
echo "MATURITY_DATE = $MATURITY_DATE"

NOW = Sun Mar  5 06:35:46 AM MST 2023
MINIMUM_ADA = 3000000
PRINCIPAL = 100
INTEREST = 2
STATUS_DATE = 2023-03-05T00:00:00
INITIAL_EXCHANGE_DATE = 2024-01-01T00:00:00
MATURITY_DATE = 2025-01-01T00:00:00


In [5]:
yaml2json << EOI > actus-pam.json
scheduleConfig:
  businessDayConvention: "NULL"
  endOfMonthConvention: "EOM"
  calendar: "NC"
maturityDate: "$MATURITY_DATE"
contractId: "0"
enableSettlement: false
initialExchangeDate: "$INITIAL_EXCHANGE_DATE"
contractRole: "RPA"
penaltyType: "O"
cycleAnchorDateOfInterestPayment: "$INITIAL_EXCHANGE_DATE"
contractType: "PAM"
notionalPrincipal: $PRINCIPAL
contractPerformance: "PF"
collateralAmount: 0
dayCountConvention: "30E360"
accruedInterest: 0
statusDate: "$STATUS_DATE"
cycleOfInterestPayment: "P1YL1"
prepaymentEffect: "N"
nominalInterestRate: $INTEREST_RATE
interestCalculationBase: "NT"
EOI
cat actus-pam.json

{"accruedInterest":0,"collateralAmount":0,"contractId":"0","contractPerformance":"PF","contractRole":"RPA","contractType":"PAM","cycleAnchorDateOfInterestPayment":"2024-01-01T00:00:00","cycleOfInterestPayment":"P1YL1","dayCountConvention":"30E360","enableSettlement":false,"initialExchangeDate":"2024-01-01T00:00:00","interestCalculationBase":"NT","maturityDate":"2025-01-01T00:00:00","nominalInterestRate":0.02,"notionalPrincipal":100,"penaltyType":"O","prepaymentEffect":"N","scheduleConfig":{"businessDayConvention":"NULL","calendar":"NC","endOfMonthConvention":"EOM"},"statusDate":"2023-03-05T00:00:00"}


Now use the `marlowe-cli template actus` command to convert the ACTUS parameters into a Marlowe contract. See the documentation for [marlowe-cli template](../../marlowe-cli/doc/template.md) for more information.

In [6]:
marlowe-cli template actus \
  --minimum-ada "$MINIMUM_ADA" \
  --party c.marlowe \
  --counter-party j.webster \
  --actus-terms-file  actus-pam.json \
  --out-contract-file actus-pam-contract.json \
  --out-state-file /dev/null

View the contract, which closely corresponds to the Blocky diagram shown above for the contract.

In [7]:
json2yaml actus-pam-contract.json

timeout: 1704067200000
timeout_continuation: close
when:
- case:
    deposits:
      negate:
        negate: 100000000
    into_account:
      role_token: c.marlowe
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      role_token: c.marlowe
  then:
    from_account:
      role_token: c.marlowe
    pay: 100000000
    then:
      timeout: 1735689600000
      timeout_continuation: close
      when:
      - case:
          deposits: 2000000
          into_account:
            role_token: j.webster
          of_token:
            currency_symbol: ''
            token_name: ''
          party:
            role_token: j.webster
        then:
          from_account:
            role_token: j.webster
          pay: 2000000
          then:
            timeout: 1735689600000
            timeout_continuation: close
            when:
            - case:
                deposits: 100000000
                into_account:
                  role_token: j.webster
                o

## Run the Contract

Running this example contract to completion involves four transactions on the blockchain:
1. Create the contract by depositing the minimum Ada required to store the contract on the blockchain.
2. The party deposits the funds for the loan, which triggers the payment of the principal to the role-payout address for the benefit of the counterparty.
3. The counterparty withdraws the principal from the role-payout address.
4. The counterparty deposits their first (and only) installment of interest, which triggers its payment to the role-payout address for the benefit of the party.
5. The party withdraws the interest from the role-payout address.
6. The counterparty deposits the principal, which triggers its payment to the role-payout address for the benefit of the party
7. The party withdraws the principal from the role-payout address.

### Transaction 1. Create the contract

We use the `marlowe create` command to build the transaction that creates the contract. This requires the following input data:
- `--core-file`: The name of Core Marlowe JSON file containing the contract to be created.
- `--min-utxo`: The amount of Ada to be deposited in the contract when it is created, so that the Cardano ledger's minimum-UTxO requirement is satisfied.
- `--change-address`: The Cardano address to which the leftover change from the transaction will be sent.
- `--manual-sign`: The name of the file that will be created to store the unsigned transaction.

The command will create the unsigned transaction the identifier for the transaction, which is also the identifier for the Marlowe contract instance. See the [help page](marlowe/create.md) for more information about this command.

In [8]:
CONTRACT_ID=$(
marlowe create \
  --core-file actus-pam-contract.json \
  --role-token-policy-id "$ROLES_CURRENCY" \
  --min-utxo "$MINIMUM_ADA" \
  --change-address "${ROLE_ADDR[c.marlowe]}" \
  --manual-sign tx-1.unsigned \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
TX_1="${CONTRACT_ID%%#1}"
echo "CONTRACT_ID = $CONTRACT_ID"
echo "TX_1 = $TX_1"

CONTRACT_ID = ace0804f10b8af1e6a70e074904e3759e982c60ccc71d7787a74270227b2de66#1
TX_1 = ace0804f10b8af1e6a70e074904e3759e982c60ccc71d7787a74270227b2de66


For security reasons, Marlowe Runtime will not sign transactions or handle signing keys. To sign the transaction here, we use the `cardano-cli transaction sign` tool, but any wallet or signing tool could be used.

In [9]:
cardano-cli transaction sign \
  --tx-body-file tx-1.unsigned \
  --out-file     tx-1.signed \
  --signing-key-file "${ROLE_SKEY[c.marlowe]}"

We now submit the transaction using Marlowe Runtime and wait for the confirmation.

In [10]:
marlowe submit tx-1.signed | jq fromjson | json2yaml

blockHeaderHash: c680e44dd0678ee05d7acf7094efb55b58676aeaf027e98f4035ec4dbd37ae1d
blockNo: 8478412
slotNo: 86458246


### View the result of creating the contract

We can now view the UTxO that holds the newly created contract. First we need to compute the address of the Marlowe contract.

In [11]:
CONTRACT_ADDR=$(marlowe-cli contract address --mainnet)
echo "$CONTRACT_ADDR"

addr1w94f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvq8evxaf


We now query the blockchain at this address, just printing this one UTxO. We see that the contract has the 3 Ada that was used to create the contract and it has a datum hash that records the contract and its state.

In [12]:
cardano-cli query utxo --mainnet --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/$TX_1/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
ace0804f10b8af1e6a70e074904e3759e982c60ccc71d7787a74270227b2de66     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "2840b4af97a9af43bf6c5475c211125b4c3100c8995f1153eb81d218ba13ef72"


View the contract on a Cardano explorer.

In [13]:
echo "https://cardanoscan.io/transaction/$TX_1?tab=utxo"

https://cardanoscan.io/transaction/ace0804f10b8af1e6a70e074904e3759e982c60ccc71d7787a74270227b2de66?tab=utxo


### Use Marlowe Runtime to query information about the contract

Now that we know the contract identifier, we can view the contract using Marlowe Runtime.

In [14]:
marlowe log --show-contract "$CONTRACT_ID"

transaction ace0804f10b8af1e6a70e074904e3759e982c60ccc71d7787a74270227b2de66 (creation)
ContractId:      ace0804f10b8af1e6a70e074904e3759e982c60ccc71d7787a74270227b2de66#1
SlotNo:          86458246
BlockNo:         8478412
BlockId:         c680e44dd0678ee05d7acf7094efb55b58676aeaf027e98f4035ec4dbd37ae1d
ScriptAddress:   addr1w94f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvq8evxaf
Marlowe Version: 1

    When [
      (Case
         (Deposit (Role "c.marlowe") (Role "c.marlowe")
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay (Role "c.marlowe")
            (Party (Role "j.webster"))
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
                  (Deposit (Role "j.webster") (Role "j.webster")
                     (Token "" "")
                     (Constant 2000000))
                  (Pay (Role "j.webster")
                     (Party (Role "c.marlowe"))
  

### Transaction 2. Party deposits loan amount

We use the `marlowe deposit` command to build the transaction that makes the first deposit for the contract. This requires the following input data:
- `--contract`: The identifier of the contract, which is the UTxO that created the contract.
- `--from-party`: The party making the deposit.
- `--to-party`: The internal account receiving the deposit.
- `--lovelace`: The amount of lovelace being deposited.
- `--change-address`: The Cardano address to which the leftover change from the transaction will be sent.
- `--manual-sign`: The name of the file that will be created to store the unsigned transaction.

The command will create the unsigned transaction the identifier for the transaction, which is also the identifier for the Marlowe contract instance.

In [15]:
TX_2=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party c.marlowe \
  --to-party c.marlowe \
  --lovelace "$((PRINCIPAL*ADA))" \
  --change-address "${ROLE_ADDR[c.marlowe]}" \
  --manual-sign tx-2.unsigned \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)

echo "TX_2 = $TX_2"

TX_2 = 4a258c3ddba68f2ea34e820bb56a03c31f0cb0f310fb515241bdd4742c4abf9a


Sign the transaction, as we did previously.

In [16]:
cardano-cli transaction sign \
  --tx-body-file tx-2.unsigned \
  --out-file     tx-2.signed \
  --signing-key-file "${ROLE_SKEY[c.marlowe]}"

Submit the transaction using Marlowe Runtime.

In [17]:
marlowe submit tx-2.signed | jq fromjson | json2yaml

blockHeaderHash: 5080340fda7fd602c9d936a3633b67c766f50d3015b3cc722ba726b3a64c0640
blockNo: 8478430
slotNo: 86458680


### View the result of depositing the funds.

Because the deposit passed through the contract, the amount of Ada in the contract hasn't changed. The new datum hash reflects the change in the state of the contract.

In [18]:
cardano-cli query utxo --mainnet --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/$TX_2/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
4a258c3ddba68f2ea34e820bb56a03c31f0cb0f310fb515241bdd4742c4abf9a     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "2f4fcb8cad5686eab7f0e17761258917996ddcc821c56687791a6d8aa9cf8ddf"


View the transaction on a Cardano explorer.

In [19]:
echo "https://cardanoscan.io/transaction/$TX_2?tab=utxo"

https://cardanoscan.io/transaction/4a258c3ddba68f2ea34e820bb56a03c31f0cb0f310fb515241bdd4742c4abf9a?tab=utxo


### Use Marlowe Runtime to Query the Contract

There are now two transactions in the contract instance's history.

In [20]:
marlowe log --show-contract "$CONTRACT_ID"

transaction ace0804f10b8af1e6a70e074904e3759e982c60ccc71d7787a74270227b2de66 (creation)
ContractId:      ace0804f10b8af1e6a70e074904e3759e982c60ccc71d7787a74270227b2de66#1
SlotNo:          86458246
BlockNo:         8478412
BlockId:         c680e44dd0678ee05d7acf7094efb55b58676aeaf027e98f4035ec4dbd37ae1d
ScriptAddress:   addr1w94f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvq8evxaf
Marlowe Version: 1

    When [
      (Case
         (Deposit (Role "c.marlowe") (Role "c.marlowe")
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay (Role "c.marlowe")
            (Party (Role "j.webster"))
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
                  (Deposit (Role "j.webster") (Role "j.webster")
                     (Token "" "")
                     (Constant 2000000))
                  (Pay (Role "j.webster")
                     (Party (Role "c.marlowe"))
  

We see that the contract has become smaller after the party made their deposit of principal.

![Contract after party's deposit of principal](actus-pam-2.png)

### Transaction 3. The counterparty withdraws the principal

The principal was deposited at Marlowe's role-payout address for the benefit of the counterparty. Now the counterparty withdraws those funds.

In [21]:
TX_3=$(
marlowe withdraw \
  --contract "$CONTRACT_ID" \
  --role j.webster \
  --change-address "${ROLE_ADDR[j.webster]}" \
  --manual-sign tx-3.unsigned \
| jq -r 'fromjson | .txId' \
)
echo "TX_3 = $TX_3"

TX_3 = ab3b74c04213c8b636cd607a1a073e1fd6f08c5bf74204bb550c490088357259


Sign the transaction, as we did previously.

In [22]:
cardano-cli transaction sign \
  --tx-body-file tx-3.unsigned \
  --out-file     tx-3.signed \
  --signing-key-file "${ROLE_SKEY[j.webster]}"

Submit the transaction using Marlowe Runtime.

In [23]:
marlowe submit tx-3.signed | jq fromjson | json2yaml

blockHeaderHash: ab1f0cc1e3b134393fabfd05b96af32473cb36ec6f1df879b4117035b020bdb7
blockNo: 8478449
slotNo: 86459052


View the transaction on a Cardano explorer.

In [24]:
echo "https://cardanoscan.io/transaction/$TX_3?tab=utxo"

https://cardanoscan.io/transaction/ab3b74c04213c8b636cd607a1a073e1fd6f08c5bf74204bb550c490088357259?tab=utxo


### Transaction 4. Counterparty repays the loan's interest

The counterparty now repays the interest.

In [25]:
TX_4=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party j.webster \
  --to-party j.webster \
  --lovelace "$((INTEREST*ADA))" \
  --change-address "${ROLE_ADDR[j.webster]}" \
  --manual-sign tx-4.unsigned \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)

echo "TX_4 = $TX_4"

TX_4 = 7e0cd07b407dcc93bee3a2a53575d4b42a8e88d02db47b9010845438ff371e1c


Sign the transaction.

In [26]:
cardano-cli transaction sign \
  --tx-body-file tx-4.unsigned \
  --out-file     tx-4.signed \
  --signing-key-file "${ROLE_SKEY[j.webster]}"

Submit the transaction using Marlowe Runtime.

In [27]:
marlowe submit tx-4.signed | jq fromjson | json2yaml

blockHeaderHash: 1760bec40a00dae77facbf21b71a0000d31c327b4697284b78b6e2b5be896f36
blockNo: 8478462
slotNo: 86459329


View the transaction on a Cardano explorer.

In [28]:
echo "https://cardanoscan.io/transaction/$TX_4?tab=utxo"

https://cardanoscan.io/transaction/7e0cd07b407dcc93bee3a2a53575d4b42a8e88d02db47b9010845438ff371e1c?tab=utxo


### Transaction 5. The party withdraws the interest

The interest was deposited at Marlowe's role-payout address for the benefit of the party. Now the party withdraws those funds.

In [29]:
TX_5=$(
marlowe withdraw \
  --contract "$CONTRACT_ID" \
  --role c.marlowe \
  --change-address "${ROLE_ADDR[c.marlowe]}" \
  --manual-sign tx-5.unsigned \
| jq -r 'fromjson | .txId' \
)
echo "TX_5 = $TX_5"

TX_5 = e256d25253a81703179f381dd014fccb0fc4d34abbebce33922f8baaaf880a71


Sign the transaction, as we did previously.

In [30]:
cardano-cli transaction sign \
  --tx-body-file tx-5.unsigned \
  --out-file     tx-5.signed \
  --signing-key-file "${ROLE_SKEY[c.marlowe]}"

Submit the transaction using Marlowe Runtime.

In [31]:
marlowe submit tx-5.signed | jq fromjson | json2yaml

blockHeaderHash: b4911bf56e3f8bdd34daf9f7956457132a6936e4654310195db0c9376540115e
blockNo: 8478464
slotNo: 86459416


View the transaction on a Cardano explorer.

In [32]:
echo "https://cardanoscan.io/transaction/$TX_5?tab=utxo"

https://cardanoscan.io/transaction/e256d25253a81703179f381dd014fccb0fc4d34abbebce33922f8baaaf880a71?tab=utxo


### Use Marlowe Runtime to query the contract

Now we see that there are five transactions in the contract's history.

In [33]:
marlowe log --show-contract "$CONTRACT_ID"

transaction ace0804f10b8af1e6a70e074904e3759e982c60ccc71d7787a74270227b2de66 (creation)
ContractId:      ace0804f10b8af1e6a70e074904e3759e982c60ccc71d7787a74270227b2de66#1
SlotNo:          86458246
BlockNo:         8478412
BlockId:         c680e44dd0678ee05d7acf7094efb55b58676aeaf027e98f4035ec4dbd37ae1d
ScriptAddress:   addr1w94f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvq8evxaf
Marlowe Version: 1

    When [
      (Case
         (Deposit (Role "c.marlowe") (Role "c.marlowe")
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay (Role "c.marlowe")
            (Party (Role "j.webster"))
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
                  (Deposit (Role "j.webster") (Role "j.webster")
                     (Token "" "")
                     (Constant 2000000))
                  (Pay (Role "j.webster")
                     (Party (Role "c.marlowe"))
  

We see that the contract has become even smaller after the counterparty made their deposit of interest.

![Contract after counterparty's deposit of interest](actus-pam-3.png)

### Transaction 6. Counterparty repays the loan's principal

The counterparty repays the principal.

In [34]:
TX_6=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party j.webster \
  --to-party j.webster \
  --lovelace "$((PRINCIPAL*ADA))" \
  --change-address "${ROLE_ADDR[j.webster]}" \
  --manual-sign tx-6.unsigned \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)

echo "TX_6 = $TX_6"

TX_6 = 4c32fdd89f29f6cf63d08b465da78141f9cab512732f4d8642f2df542d10071c


Sign the transaction.

In [35]:
cardano-cli transaction sign \
  --tx-body-file tx-6.unsigned \
  --out-file     tx-6.signed \
  --signing-key-file "${ROLE_SKEY[j.webster]}"

Submit the transaction using Marlowe Runtime.

In [36]:
marlowe submit tx-6.signed | jq fromjson | json2yaml

blockHeaderHash: 2b0600b56ff0f9e02dfae617e4182a7bc0e71078b2de8556794bc912ed274534
blockNo: 8478476
slotNo: 86459663


View the transaction on a Cardano explorer.

In [37]:
echo "https://cardanoscan.io/transaction/$TX_6?tab=utxo"

https://cardanoscan.io/transaction/4c32fdd89f29f6cf63d08b465da78141f9cab512732f4d8642f2df542d10071c?tab=utxo


### Transaction 7. The party withdraws the principal

The principal was deposited at Marlowe's role-payout address for the benefit of the party. Now the party withdraws those funds.

In [38]:
TX_7=$(
marlowe withdraw \
  --contract "$CONTRACT_ID" \
  --role c.marlowe \
  --change-address "${ROLE_ADDR[c.marlowe]}" \
  --manual-sign tx-7.unsigned \
| jq -r 'fromjson | .txId' \
)
echo "TX_7 = $TX_7"

TX_7 = c26da1e092d792f21390c9fd8be4984dec9e5e32b770302c8f562c210d498172


Sign the transaction, as we did previously.

In [39]:
cardano-cli transaction sign \
  --tx-body-file tx-7.unsigned \
  --out-file     tx-7.signed \
  --signing-key-file "${ROLE_SKEY[c.marlowe]}"

Submit the transaction using Marlowe Runtime.

In [40]:
marlowe submit tx-7.signed | jq fromjson | json2yaml

blockHeaderHash: 22d711a6713a9213802d9866f12101724df74d8d3c8c90a4de08a2a53afd00d6
blockNo: 8478483
slotNo: 86459737


View the transaction on a Cardano explorer.

In [41]:
echo "https://cardanoscan.io/transaction/$TX_7?tab=utxo"

https://cardanoscan.io/transaction/c26da1e092d792f21390c9fd8be4984dec9e5e32b770302c8f562c210d498172?tab=utxo


### Use Marlowe Runtime to view the full history of the contract

We now see the final transaction in the contract's history.

In [42]:
marlowe log --show-contract "$CONTRACT_ID"

transaction ace0804f10b8af1e6a70e074904e3759e982c60ccc71d7787a74270227b2de66 (creation)
ContractId:      ace0804f10b8af1e6a70e074904e3759e982c60ccc71d7787a74270227b2de66#1
SlotNo:          86458246
BlockNo:         8478412
BlockId:         c680e44dd0678ee05d7acf7094efb55b58676aeaf027e98f4035ec4dbd37ae1d
ScriptAddress:   addr1w94f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvq8evxaf
Marlowe Version: 1

    When [
      (Case
         (Deposit (Role "c.marlowe") (Role "c.marlowe")
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay (Role "c.marlowe")
            (Party (Role "j.webster"))
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
                  (Deposit (Role "j.webster") (Role "j.webster")
                     (Token "" "")
                     (Constant 2000000))
                  (Pay (Role "j.webster")
                     (Party (Role "c.marlowe"))
  